In [1]:
##########################################
# 1. STOCK MARKET DATA
##########################################

%reset -f
import pandas as pd

# Load data
df = pd.read_csv("StockMarket.csv")

# Select variables
df = df[['ticker', 'year', 'month', 'ret', 'prc','shrout']]

print(df)

       ticker  year  month       ret         prc  shrout
0        SUNW  1987      5  0.208333   43.500000   33414
1        SUNW  1987      6 -0.063218   40.750000   34096
2        SUNW  1987      7 -0.150307   34.625000   34096
3        SUNW  1987      8  0.119134   38.750000   34096
4        SUNW  1987      9 -0.064516   36.250000   34177
...       ...   ...    ...       ...         ...     ...
272184   TSLA  2019      8 -0.066222  225.610001  179127
272185   TSLA  2019      9  0.067639  240.869995  180000
272186   TSLA  2019     10  0.307427  314.920013  180245
272187   TSLA  2019     11  0.047695  329.940002  180245
272188   TSLA  2019     12  0.267897  418.329987  181062

[272189 rows x 6 columns]


In [ ]:
# 1. Select the stock of Apple in December 2019
apple = df.loc[
    (df['ticker']=="????") # What is Apple's ticker?
    & (df['year']==2019) & (df['month']==12),
    ['ticker', 'year', 'month', 'prc', 'shrout', 'ret']]

In [ ]:
# At the end of Dec 2019, how much is Apple's:
#   2. stock price
#   3. number of shares outstanding
#   4. market capitalization


In [1]:
##########################################
# 2. SHARPE RATIO
##########################################

%reset -f
import pandas as pd

# Stock market data
df = pd.read_csv("StockMarket.csv")

# Select variables
df = df[['ticker', 'year', 'month', 'ret', 'rf']]

# Create a date in format year-month
df['date'] = df['year'].astype(str) + "-" + df['month'].astype(str).str.zfill(2)

# Filter for Jan 2010 – Dec 2019
df = df[(df['date'] >= '2010-01') & (df['date'] <= '2019-12')]

print(df)

       ticker  year  month       ret      rf     date
549      ORCL  2010      1 -0.057888  0.0000  2010-01
550      ORCL  2010      2  0.068951  0.0000  2010-02
551      ORCL  2010      3  0.043002  0.0001  2010-03
552      ORCL  2010      4  0.008071  0.0001  2010-04
553      ORCL  2010      5 -0.127477  0.0001  2010-05
...       ...   ...    ...       ...     ...      ...
272184   TSLA  2019      8 -0.066222  0.0016  2019-08
272185   TSLA  2019      9  0.067639  0.0018  2019-09
272186   TSLA  2019     10  0.307427  0.0016  2019-10
272187   TSLA  2019     11  0.047695  0.0012  2019-11
272188   TSLA  2019     12  0.267897  0.0014  2019-12

[67028 rows x 6 columns]


In [43]:
### CONSTRUCT A PORTFOLIO ###

# Portfolio: list of stocks
portfolio = ['MSFT', 'DIS', 'JPM', 'F', 'KO'] # Microsoft, Disney, JP Morgan, Ford, Coca-Cola

# Filter for the selected stocks
df_sub = df[df['ticker'].isin(portfolio)].copy()

# Pivot to have tickers as columns, returns as values
df_p = df_sub.pivot(index='date', columns='ticker', values='ret')

# Equal-weighted portfolio return = simple average across the 5 stocks
df_p['ret_p'] = df_p.mean(axis=1)

# Risk-free return
df_rf = df_sub.drop_duplicates(['date'])[['date','rf']].set_index('date')['rf']

# Merge everything
df_p = df_p[['ret_p']].join(df_rf)

print(df_p)

            ret_p      rf
date                     
2010-01 -0.037548  0.0000
2010-02  0.042366  0.0000
2010-03  0.065509  0.0001
2010-04  0.011625  0.0001
2010-05 -0.090306  0.0001
...           ...     ...
2019-08 -0.014016  0.0016
2019-09  0.004895  0.0018
2019-10  0.010238  0.0016
2019-11  0.064808  0.0012
2019-12  0.024539  0.0014

[120 rows x 2 columns]


In [ ]:
### PORTFOLIO'S SHARPE RATIO ###

# Portfolio excess return (portfolio return minus risk-free rate)
df_p['retrf_p'] = ????????

# Mean excess return (annualized)
mean_ret = ??????????
print("Mean excess return (% per year):", mean_ret)

# S.D. excess return (annualized)
std_ret = ?????????
print("S.D. excess return (% per year):", std_ret)

# Sharpe ratio (annualized)
sharpe = ??????????
print("Sharpe ratio (annualized):", sharpe)

In [47]:
### INDIVIDUAL STOCKS' SHARPE RATIO ###

# Individual stocks' excess return
df_sub['retrf'] = df_sub['ret'] - df_sub['rf']

# Function to compute annualized Sharpe
def sharpe_ratio(x):
    mean_ret = x.mean() # monthly
    std_ret = x.std() # monthly
    sharpe_annualized = mean_ret / std_ret * 12**.5 # multiple by square root of 12 to annualize the Sharpe ratio
    return sharpe_annualized

# Group by ticker and apply
sharpe_by_ticker = df_sub.groupby('ticker')['retrf'].apply(sharpe_ratio)

print(sharpe_by_ticker)

ticker
DIS     0.872093
F       0.222331
JPM     0.703353
KO      0.766756
MSFT    0.976920
Name: retrf, dtype: float64


In [48]:
###############################
# 3. CAPM
###############################

%reset -f
import pandas as pd
import statsmodels.api as sm

# Stock market data
df = pd.read_csv("StockMarket.csv")

# Select variables
df = df[['ticker', 'year', 'month', 'ret', 'mkt', 'rf']]

# Select Ford stock
ford = df[df['ticker'] == "F"].copy()
print(ford)

      ticker  year  month       ret     mkt      rf
87879      F  1980      1  0.085938  0.0631  0.0080
87880      F  1980      2 -0.096296 -0.0033  0.0089
87881      F  1980      3 -0.045082 -0.1169  0.0121
87882      F  1980      4 -0.128755  0.0523  0.0126
87883      F  1980      5  0.005128  0.0607  0.0081
...      ...   ...    ...       ...     ...     ...
88354      F  2019      8 -0.037775 -0.0242  0.0016
88355      F  2019      9 -0.001091  0.0161  0.0018
88356      F  2019     10 -0.045851  0.0222  0.0016
88357      F  2019     11  0.054715  0.0399  0.0012
88358      F  2019     12  0.026490  0.0291  0.0014

[480 rows x 6 columns]


In [54]:
### ESTIMATE CAPM REGRESSION ###

# Dependent variable: Ford's excess return (r_i-r_f)
ford['retrf'] = ford['ret'] - ford['rf']
y = ford['retrf']

# Regressors: market excess return (r_m-r_f) and a constant (alpha)
ford['constant'] = 1
ford['mktrf'] = ford['mkt'] - ford['rf']
X = ford[['mktrf', 'constant']]

# Run OLS
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  retrf   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     159.8
Date:                Sun, 05 Oct 2025   Prob (F-statistic):           8.33e-32
Time:                        14:14:07   Log-Likelihood:                 425.52
No. Observations:                 480   AIC:                            -847.0
Df Residuals:                     478   BIC:                            -838.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mktrf          1.3121      0.104     12.642      0.0

In [60]:
# Average return of Ford and market (per year)
print( "Average return:", ford['ret'].mean()*12)
print( "Average return:", ford['mkt'].mean()*12)

Average return: 0.15949572323297617
Average return: 0.12299500183675002
